## Load packages

In [2]:
import os
from pathlib import Path
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
from osgeo import gdal, ogr, osr
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio

## Setup

In [3]:
gdal.UseExceptions()
path_osgeo_utils= "C:\\NMBU\\Miniconda3\\envs\\deep_learning\\Lib\\site-packages\\GDAL-3.7.0-py3.11-win-amd64.egg-info\\scripts"

tile_size_m = 10 # tile size in m
buffer_size_m = 0 # tile buffer size_m (used to produce overlapping tiles)

path_data = "..\\data"
orthos = os.listdir(path_data + "\\orthomosaics\\test_data")
dir_drone_acquisitions = path_data + "/map_data/drone_acquisitions.geojson"

In [4]:
footprints= gpd.read_file(dir_drone_acquisitions)

## Retile orthos

Loop though orthos and retile.

In [6]:
for o in orthos:
    input_ortho_path = f"{path_data}\\orthomosaics\\test_data\\{o}"
    ortho_name = os.path.splitext(os.path.basename(input_ortho_path))[0]
    output_tiles_dir = f"{path_data}\\tiles\\{tile_size_m}m_{ortho_name}"
    if not os.path.exists(output_tiles_dir):
        os.makedirs(output_tiles_dir)

    # Collect metadata
    src_ds = gdal.Open(input_ortho_path) # reads in the orthomosaic
    _, xres, _, _, _, yres  = src_ds.GetGeoTransform()
    proj = osr.SpatialReference(wkt = src_ds.GetProjection())
    EPSG_code = proj.GetAttrValue('AUTHORITY',1)
    n_bands =src_ds.RasterCount

    # Compute size of tile and buffer
    tile_size_px = round(tile_size_m / abs(xres)) # calculate the tile size in pixels
    buffer_size_px = round(buffer_size_m / abs(xres)) # calculate the buffer size in pixels

    # define name for output tile index shapefile
    tileIndex_name = ortho_name + "_tile_index"

    # Run gdal_retile.py using CLI (can take some minutes) 
    command_retile = "python " + path_osgeo_utils + "/gdal_retile.py -targetDir " + output_tiles_dir + " " + input_ortho_path + " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of GTiff -tileIndex "+ tileIndex_name + " -tileIndexField ID"
    print(os.popen(command_retile).read()) 

    # Select only tiles within footprint
    footprint_ortho = footprints[footprints['filename'] == ortho_name]
    footprint_ortho_UU = footprint_ortho.geometry.unary_union

    # Load tiles shapefile
    tiles = gpd.read_file(output_tiles_dir+ "/"+ortho_name+"_tile_index.shp")
    tiles= tiles.to_crs(EPSG_code)

    # Select all tiles that are within the boundary polygon
    tiles_in = tiles[tiles.geometry.within(footprint_ortho_UU)]

    # Select all tiles that are not within the boundary polygon
    tiles_out= tiles.loc[~tiles['ID'].isin(tiles_in['ID']) ]
    print(str(len(tiles_out))+" tiles to be deleted")
    print(str(len(tiles_in))+" tiles remaining")

    # delete tiles that are not within the footprint
    gtiffs_delete=[output_tiles_dir + "/"+sub  for sub in tiles_out['ID']] 
    for f in gtiffs_delete:
        if os.path.exists(f):
            os.remove(f)

0...10...20...30...40...50...60...70...80...90...100 - done.

903 tiles to be deleted
337 tiles remaining
0...10...20...30...40...50...60...70...80...90...100 - done.

714 tiles to be deleted
470 tiles remaining
0...10...20...30...40...50...60...70...80...90...100 - done.

283 tiles to be deleted
177 tiles remaining
0...10...20...30...40...50...60...70...80...90...100 - done.

194 tiles to be deleted
158 tiles remaining
